# Protein sequence playing around
### Continued

In [1]:
import numpy as np
import requests
import re
import xml.etree.ElementTree as ET

import matplotlib.pyplot as plt

### Load data
Sequences were previously taken from the RCSB website

In [2]:
import pickle
with open ('fastas', 'rb') as read:
    all_seq = pickle.load(read)

Full vocabulary

In [4]:
unique = set()
for s in all_seq:
    for a in s:
        unique.update(a)
''.join(unique)

'IWPulCREDMQSNGVFYHLnKTXAU'

In [39]:
# conversion maps
n_to_aa = {i:v for i,v in enumerate(unique)}
aa_to_n = {v:i for i,v in enumerate(unique)}

Truncating short sequences

In [5]:
X_raw = [x for x in all_seq if len(x)>50]
len(X_raw)

29778

### Model

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.preprocessing import sequence

Using TensorFlow backend.


In [7]:
# vocabulary (number of one-hot-encoded columns)
voc = len(unique)
# window size ('memory')
win = 25

model = Sequential()
model.add(LSTM((64),input_shape=(win, voc),return_sequences=True))
model.add(LSTM((128),input_shape=(win, voc),return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(256,activation='relu'))   
model.add(Dense(voc,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 25, 64)            23040     
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               98816     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 25)                6425      
Total params: 161,305
Trainable params: 161,305
Non-trainable params: 0
_______________

In [8]:
# prepare X and y. X_total is a tuple of 'x' and 'y' 
# everything is essentially taken from the same sequences 'X_raw'
X_total = []
for x in X_raw:
    X_total.extend([(x[i:i+win],x[i+win]) for i in range(len(x)-(win))])
len(X_total)

7648920

In [34]:
# input and labels generator
def one_hot_generator(X_all,batch_size,window,vocabulary,iterations=10):
    # run indefinitely
    while (True):
        # generate new indices
        indices = np.random.choice(range(len(X_all)),size=batch_size*iterations,replace=False)
        for i in range(iterations):
            # subset of indices, generate one-hot-encoded X and y
            total = np.array(X_all)[indices[i*batch_size:(i+1)*batch_size]]

            X = np.zeros((batch_size,window,vocabulary),dtype=int)
            y = np.zeros((batch_size,vocabulary),dtype=int)

            for i in range(batch_size):
                xy = total[i]
                X[i][range(win),[aa_to_n[i] for i in xy[0]]] = 1
                y[i][aa_to_n[xy[1]]] = 1  

            yield X,y

### Train model

In [42]:
model.fit_generator(one_hot_generator(X_total,25000,win,voc),steps_per_epoch=10,epochs=10)

Epoch 1/10
10/10 [==============================] - 204s 20s/step - loss: 2.8126 - acc: 0.1041
Epoch 2/10
10/10 [==============================] - 189s 19s/step - loss: 2.8121 - acc: 0.1045
Epoch 3/10
10/10 [==============================] - 201s 20s/step - loss: 2.8115 - acc: 0.1056
Epoch 4/10
10/10 [==============================] - 190s 19s/step - loss: 2.8107 - acc: 0.1055
Epoch 5/10
10/10 [==============================] - 191s 19s/step - loss: 2.8111 - acc: 0.1061
Epoch 6/10
10/10 [==============================] - 204s 20s/step - loss: 2.8109 - acc: 0.1058
Epoch 7/10
10/10 [==============================] - 191s 19s/step - loss: 2.8098 - acc: 0.1058
Epoch 8/10
10/10 [==============================] - 187s 19s/step - loss: 2.8103 - acc: 0.1061
Epoch 9/10
10/10 [==============================] - 188s 19s/step - loss: 2.8079 - acc: 0.1066
Epoch 10/10
10/10 [==============================] - 188s 19s/step - loss: 2.8074 - acc: 0.1074


### Evaluate model

In [43]:
model.evaluate_generator(one_hot_generator(X_total,10000,win,voc),steps=10)

[2.8067962169647216, 0.10736999958753586]